### Switching probability in first 100 trials
- Unstructured sessions showed higher alpha_chosen and lower alpha_perseverance compared to structured sessions, suggesting animals are more likely to switch in the unstructured sessions.

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects

exps = mab_subjects.unstruc.allsess + mab_subjects.struc.allsess

swp_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    mab = exp.mab.keep_by_trials(min_trials=100, clip_max=100)
    switch_prob = mab.get_switch_prob()
    print(f"Switch prob: {switch_prob}")

    df = pd.DataFrame(
        {
            "switch_prob": [switch_prob],
            "name": exp.sub_name,
            "grp": "struc" if mab.is_structured else "unstruc",
        }
    )

    swp_df.append(df)

swp_df = pd.concat(swp_df, ignore_index=True)
mab_subjects.GroupData().save(swp_df, "switch_prob_100trials")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statannotations.Annotator import Annotator
from statplot_utils import stat_kw

fig = plotting.Fig(1, 3, size=(11, 3), num=1)

grpdata = mab_subjects.GroupData()
df = grpdata.switch_prob_100trials

ax = fig.subplot(fig.gs[0])
# ax.axhline(0, color="gray", lw=0.8, zorder=0)

plot_kw = dict(data=df, x="grp", y="switch_prob", hue="grp", ax=ax)
sns.stripplot(
    palette=["gray", "gray"],
    edgecolor="white",
    # facecolor=(0, 0, 0, 0),
    # alpha=0.4,
    # errorbar="se",
    **plot_kw,
)
sns.barplot(
    palette="husl",
    # edgecolor="0.5",
    # facecolor=(0, 0, 0, 0),
    alpha=0.4,
    errorbar="se",
    **plot_kw,
)


orders = ["unstruc", "struc"]
pairs = [(("unstruc"), ("struc"))]
annotator = Annotator(pairs=pairs, order=orders, **plot_kw)
annotator.configure(test="Kruskal", **stat_kw, color="k", verbose=True)
annotator.apply_and_annotate()
annotator.reset_configuration()

ax.set_title("Switch probability")
# ax.legend("")

### Switching probability as a function of trials within session
- Check if switch probability curves for unstructured and structured sessions have same decay and tail

In [73]:
import numpy as np
import pandas as pd
import mab_subjects
from scipy.ndimage import gaussian_filter1d


exps = mab_subjects.unstruc.allsess + mab_subjects.struc.allsess

swp_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    mab = exp.mab.keep_by_trials(min_trials=100, clip_max=100)
    switch_prob = mab.get_switch_prob_by_trial()
    switch_prob_smooth = gaussian_filter1d(switch_prob, sigma=1)

    df = pd.DataFrame(
        {
            "trial_id": np.arange(len(switch_prob)) + 1,
            "switch_prob": switch_prob,
            "switch_prob_smooth": switch_prob_smooth,
            "name": exp.sub_name,
            "grp": "struc" if mab.is_structured else "unstruc",
        }
    )

    swp_df.append(df)

swp_df = pd.concat(swp_df, ignore_index=True)
mab_subjects.GroupData().save(swp_df, "switch_prob_by_trial_100trials")

AggroExp1UnStructured
AuromaExp1Unstructured
BratExp1Unstructured
GronckleExp2Unstructured
GrumpExp1Unstructured
ToothlessExp2Unstructured
BewilderbeastExp1Structured
BuffalordExp1Structured
GronckleExp1Structured
GrumpExp2Structured
ToothlessExp1Structured
switch_prob_by_trial_100trials saved


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statannotations.Annotator import Annotator
from statplot_utils import stat_kw

fig = plotting.Fig(1, 3, size=(11, 3), num=1)

grpdata = mab_subjects.GroupData()
df = grpdata.switch_prob_by_trial_100trials

ax = fig.subplot(fig.gs[0])
# ax.axhline(0, color="gray", lw=0.8, zorder=0)

plot_kw = dict(data=df, x="trial_id", y="switch_prob", hue="grp", ax=ax)
sns.lineplot(
    palette="husl",
    errorbar="se",
    err_kws={"edgecolor": None},
    **plot_kw,
)

orders = np.arange(1, 100)
pairs = [((_, "unstruc"), (_, "struc")) for _ in orders[::10]]
annotator = Annotator(pairs=pairs, order=orders, **plot_kw)
annotator.configure(test="Kruskal", **stat_kw, color="k", verbose=True)
annotator.apply_and_annotate()
annotator.reset_configuration()

ax_ = fig.subplot(fig.gs[1])
# ax.axhline(0, color="gray", lw=0.8, zorder=0)

plot_kw = dict(data=df, x="trial_id", y="switch_prob_smooth", hue="grp", ax=ax_)
sns.lineplot(
    palette="husl",
    errorbar="se",
    err_kws={"edgecolor": None},
    **plot_kw,
)


ax.set_title("Switch probability")
# ax.legend("")
ax.set_xlim(1, 100)

p-value annotation legend:
      ns: 5.00e-02 < p <= 1.00e+00
       *: p <= 5.00e-02

11_unstruc vs. 11_struc: Kruskal-Wallis independent samples (pairwise between groups), P_val:2.733e-01 Stat=1.200e+00
21_unstruc vs. 21_struc: Kruskal-Wallis independent samples (pairwise between groups), P_val:8.551e-01 Stat=3.333e-02
31_unstruc vs. 31_struc: Kruskal-Wallis independent samples (pairwise between groups), P_val:7.150e-01 Stat=1.333e-01
1_unstruc vs. 1_struc: Kruskal-Wallis independent samples (pairwise between groups), P_val:4.652e-01 Stat=5.333e-01
41_unstruc vs. 41_struc: Kruskal-Wallis independent samples (pairwise between groups), P_val:2.733e-01 Stat=1.200e+00
51_unstruc vs. 51_struc: Kruskal-Wallis independent samples (pairwise between groups), P_val:4.652e-01 Stat=5.333e-01
61_unstruc vs. 61_struc: Kruskal-Wallis independent samples (pairwise between groups), P_val:1.003e-01 Stat=2.700e+00
71_unstruc vs. 71_struc: Kruskal-Wallis independent samples (pairwise between groups), P_

(1.0, 100.0)

c:\Users\asheshlab\miniconda3\envs\data_analysis\Lib\site-packages\ipykernel\eventloops.py:145: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or Axes decorations smaller.
  el.exec() if hasattr(el, "exec") else el.exec_()
